In [1]:
import pandas as pd
import requests
import secret

In [2]:
API_KEY = ''
#https://learndataanalysis.org/google-geocoding-api-tutorial-for-python/
def getLatLng(address):
    params = {
        'key': secret.REACT_APP_GOOGLE_MAPS_API_KEY,
        'address': address.replace(' ', '+')
    }

    api = 'https://maps.googleapis.com/maps/api/geocode/json?'
    response = requests.get(api, params=params)
    data = response.json()
    if data['status'] == 'OK':
        print(f"Processed : {address}")
        result = data['results'][0]
        location = result['geometry']['location']
        return (location['lat'], location['lng'])
    else:
        #if we cannot find then do this
        return ("NaN", "NaN")

In [3]:
df = pd.read_csv ("covid_lab_unclean.csv", index_col = False)
#filter out unnamed columns
df = df.loc[:, ~df.columns.str.contains('^Unnamed')]

In [4]:
def cleanMapAddress (x):
    x = str(x)
    x = x.strip()
    if x[-1] == ",":
        return x[:-1]
    else:
        return x


In [5]:
def replaceEmptyString (x):
    if x == '':
        return "N/A"

In [6]:
def cleanEmail (email):
    result = ""
    for i in email:
        if i == " ":
            continue
        result += i
    return result

In [7]:
def formatColumnNames (colname):
    result = ""
    colname.strip()
    l = colname.split(" ")
    for i in l:
        result += i.lower() + "_"
    return result[:-1]

In [8]:
df.columns = [formatColumnNames(x) for x in df.columns]

In [9]:
df.head()

,region,province,municipality_/_city,name_of_facility,ownership_(public_/_private),type_of_testing,license_validity
0,NCR,Metro Manila,Muntilupa City,Asian Hospital and Medical Center,PRIVATE,rRT PCR,01/01/22 - 12/31/22
1,NCR,Metro Manila,City of Manila,Chinese General Hospital,PRIVATE,rRT PCR,01/08/22 - 12/31/22
2,NCR,Metro Manila,Mandaluyong City,Detoxicare Molecular Diagnostics Laboratory,PRIVATE,rRT PCR,01/01/22 - 12/31/22
3,NCR,Metro Manila,Caloocan City,Dr. Jose N. Rodriguez Memorial Hospital and Sa...,PUBLIC,Cartridge - Based PCR,01/01/22 - 12/31/22
4,NCR,Metro Manila,Caloocan City,Dr. Jose N. Rodriguez Memorial Hospital and Sa...,PUBLIC,rRT PCR,01/01/22 - 12/31/22


In [10]:
df['index'] = df.index
df['current_distance'] = "Click the compass to find out!"
df['gmaps_url'] = "https://www.google.com/maps/dir/?api=1"
df.rename(columns = {'municipality_/_city':'city'}, inplace = True)
df.rename(columns = {'ownership_(public_/_private)':'ownership'}, inplace = True)

In [11]:
df.head()

,region,province,city,name_of_facility,ownership,type_of_testing,license_validity,index,current_distance,gmaps_url
0,NCR,Metro Manila,Muntilupa City,Asian Hospital and Medical Center,PRIVATE,rRT PCR,01/01/22 - 12/31/22,0,Click the compass to find out!,https://www.google.com/maps/dir/?api=1
1,NCR,Metro Manila,City of Manila,Chinese General Hospital,PRIVATE,rRT PCR,01/08/22 - 12/31/22,1,Click the compass to find out!,https://www.google.com/maps/dir/?api=1
2,NCR,Metro Manila,Mandaluyong City,Detoxicare Molecular Diagnostics Laboratory,PRIVATE,rRT PCR,01/01/22 - 12/31/22,2,Click the compass to find out!,https://www.google.com/maps/dir/?api=1
3,NCR,Metro Manila,Caloocan City,Dr. Jose N. Rodriguez Memorial Hospital and Sa...,PUBLIC,Cartridge - Based PCR,01/01/22 - 12/31/22,3,Click the compass to find out!,https://www.google.com/maps/dir/?api=1
4,NCR,Metro Manila,Caloocan City,Dr. Jose N. Rodriguez Memorial Hospital and Sa...,PUBLIC,rRT PCR,01/01/22 - 12/31/22,4,Click the compass to find out!,https://www.google.com/maps/dir/?api=1


In [12]:
df.head()

,region,province,city,name_of_facility,ownership,type_of_testing,license_validity,index,current_distance,gmaps_url
0,NCR,Metro Manila,Muntilupa City,Asian Hospital and Medical Center,PRIVATE,rRT PCR,01/01/22 - 12/31/22,0,Click the compass to find out!,https://www.google.com/maps/dir/?api=1
1,NCR,Metro Manila,City of Manila,Chinese General Hospital,PRIVATE,rRT PCR,01/08/22 - 12/31/22,1,Click the compass to find out!,https://www.google.com/maps/dir/?api=1
2,NCR,Metro Manila,Mandaluyong City,Detoxicare Molecular Diagnostics Laboratory,PRIVATE,rRT PCR,01/01/22 - 12/31/22,2,Click the compass to find out!,https://www.google.com/maps/dir/?api=1
3,NCR,Metro Manila,Caloocan City,Dr. Jose N. Rodriguez Memorial Hospital and Sa...,PUBLIC,Cartridge - Based PCR,01/01/22 - 12/31/22,3,Click the compass to find out!,https://www.google.com/maps/dir/?api=1
4,NCR,Metro Manila,Caloocan City,Dr. Jose N. Rodriguez Memorial Hospital and Sa...,PUBLIC,rRT PCR,01/01/22 - 12/31/22,4,Click the compass to find out!,https://www.google.com/maps/dir/?api=1


In [13]:
df.columns

Index(['region', 'province', 'city', 'name_of_facility', 'ownership',
       'type_of_testing', 'license_validity', 'index', 'current_distance',
       'gmaps_url'],
      dtype='object')

In [14]:
df['city'].fillna('', inplace = True)
df['region'].fillna('', inplace = True)
df['province'].fillna('', inplace = True)

In [15]:
df["mapping_address"] = df['province'] + " " + df['region'] + " " + df['city'] + " " + df["name_of_facility"]

In [16]:
df['city'].map(lambda x : replaceEmptyString(x))
df['region'].map(lambda x : replaceEmptyString(x))
df['province'].map(lambda x : replaceEmptyString(x))

0      None
1      None
2      None
3      None
4      None
       ... 
335    None
336    None
337    None
338    None
339    None
Name: province, Length: 340, dtype: object

In [17]:
#fill back with "NaN"
df.fillna ("N/A", inplace = True)

In [18]:
df.head()

,region,province,city,name_of_facility,ownership,type_of_testing,license_validity,index,current_distance,gmaps_url,mapping_address
0,NCR,Metro Manila,Muntilupa City,Asian Hospital and Medical Center,PRIVATE,rRT PCR,01/01/22 - 12/31/22,0,Click the compass to find out!,https://www.google.com/maps/dir/?api=1,Metro Manila NCR Muntilupa City Asian Hospital...
1,NCR,Metro Manila,City of Manila,Chinese General Hospital,PRIVATE,rRT PCR,01/08/22 - 12/31/22,1,Click the compass to find out!,https://www.google.com/maps/dir/?api=1,Metro Manila NCR City of Manila Chinese Genera...
2,NCR,Metro Manila,Mandaluyong City,Detoxicare Molecular Diagnostics Laboratory,PRIVATE,rRT PCR,01/01/22 - 12/31/22,2,Click the compass to find out!,https://www.google.com/maps/dir/?api=1,Metro Manila NCR Mandaluyong City Detoxicare M...
3,NCR,Metro Manila,Caloocan City,Dr. Jose N. Rodriguez Memorial Hospital and Sa...,PUBLIC,Cartridge - Based PCR,01/01/22 - 12/31/22,3,Click the compass to find out!,https://www.google.com/maps/dir/?api=1,Metro Manila NCR Caloocan City Dr. Jose N. Rod...
4,NCR,Metro Manila,Caloocan City,Dr. Jose N. Rodriguez Memorial Hospital and Sa...,PUBLIC,rRT PCR,01/01/22 - 12/31/22,4,Click the compass to find out!,https://www.google.com/maps/dir/?api=1,Metro Manila NCR Caloocan City Dr. Jose N. Rod...


In [19]:
df['lat'] = df["mapping_address"].map (lambda x : getLatLng(x)[0])
df['lng'] = df["mapping_address"].map (lambda x : getLatLng(x)[1])

Processed : Metro Manila NCR Muntilupa City Asian Hospital and Medical Center
Processed : Metro Manila NCR City of Manila Chinese General Hospital
Processed : Metro Manila NCR Mandaluyong City Detoxicare Molecular Diagnostics Laboratory
Processed : Metro Manila NCR Caloocan City Dr. Jose N. Rodriguez Memorial Hospital and Sanitarium (Tala)
Processed : Metro Manila NCR Caloocan City Dr. Jose N. Rodriguez Memorial Hospital and Sanitarium (Tala)
Processed : Metro Manila NCR Quezon City Lung Center of the Philippines (LCP)
Processed : Metro Manila NCR Quezon City Lung Center of the Philippines (GeneXpert)
Processed : Metro Manila NCR Makati City Makati Medical Center
Processed : Metro Manila NCR Marikina City Marikina Molecular Diagnostic laboratory
Processed : Metro Manila NCR Quezon City Philippine Genome Center UP-Diliman (NHB)
Processed : Metro Manila NCR Mandaluyong City Philippine Red Cross - National Blood Center
Processed : Metro Manila NCR Mandaluyong City Philippine Red Cross Log

Processed : Metro Manila NCR Marikina City Marikina Valley Medical Center
Processed : Metro Manila NCR Taguig City Medical Center of Taguig City, Inc.
Processed : Metro Manila NCR Parañaque City Medical Center Paranaque, Inc.
Processed : Metro Manila NCR Quezon City Best Diagnostic Corporation
Processed : Metro Manila NCR Quezon City Hi-Precision Diagnostic Center
Processed : Metro Manila NCR Quezon City Kairos Diagnostics Laboratory
Processed : Metro Manila NCR Pasay City Philippine Airport Diagnostic Laboratory
Processed : Metro Manila NCR Quezon City National Childrens Hospital
Processed : Metro Manila NCR City of Manila El Roi Medical Clinic & Diagnostic Center Inc.
Processed : Metro Manila NCR Quezon City Be Safe MD. Inc
Processed : Metro Manila NCR City of Manila Medical Center Manila, Inc
Processed : Metro Manila NCR City of Manila Dr. Jose Fabella Memorial Hospital
Processed : Metro Manila NCR City of Manila Dr. Jose Fabella Memorial Hospital
Processed : Metro Manila NCR Mandal

Processed : Cavite Region 4A Dasmarinas City City of Dasmarinas Molecular Diagnostic Laboratory
Processed : Quezon Region 4A Municipality of Mauban Mauban Municipal Health Office Molecular Laboratory
Processed : Laguna Region 4A Sta. Rosa City The Medical City South Luzon
Processed : Rizal Region 4A Municipality of Cainta Breeze Medical Services
Processed : Batangas Region 4A Lipa City Lipa Medix Medical Center
Processed : Cavite Region 4A Bacoor City St. Dominic Medical Center, Inc.
Processed : Laguna Region 4A City of Biñan UPH - Dr. Jose G. Tamayo Medical Center
Processed : Quezon Region 4A Lucena City Mount Carmel Diocesan General Hospital
Processed : Laguna Region 4A City of Biñan RTSD - Biñan Molecular Diagnostic Laboratory
Processed : Palawan Region 4B Puerto Princesa City Ospital ng Palawan
Processed : Oriental Mindoro Region 4B Calapan City Oriental Mindoro Provincial Hospital
Processed : Palawan Region 4B Municipality of Culion Culion Sanitarium and General Hospital
Processed

Processed : Davao Del Sur Region 11 Davao City Southern Philippines Medical Center (SPMC)
Processed : Davao Del Sur Region 11 Davao City Davao One World Diagnostic Center Incorporated
Processed : Davao Del Norte Region 11 Tagum City Davao Regional Medical Center
Processed : Davao Del Sur Region 11 Davao City Davao Doctors Hospital
Processed : Davao Del Sur Region 11 Davao City Southern Philippines Medical Center (SPMC-SNL-MBL)
Processed : Davao Del Sur Region 11 Davao City Davao International Airport Laboratory
Processed : Davao Del Sur Region 11 Davao City PRO 11 RT-PCR Molecular Laboratory
Processed : Davao Del Norte Region 11 Tagum City Davao Regional Medical Center
Processed : Davao De Oro Region 11 Municipality of Montevista Davao de Oro Provincial Hospital
Processed : Davao Del Sur Region 11 Davao City Vitalab Molecular Laboratory
Processed : Davao Del Sur Region 11 Davao City Safeguard DNA Diagnostics, Inc.
Processed : Davao Del Sur Region 11 Davao City Los Amigos Davao Laborato

Processed : Metro Manila NCR City of Manila Philippine General Hospital (PGH)
Processed : Metro Manila NCR City of Manila Manila Diagnostic Center for OFW Inc.
Processed : Metro Manila NCR Parañaque City Interpharma Solutions Philippines, Inc.
Processed : Metro Manila NCR City of Manila El Roi Medical Clinic & Diagnostic Center Inc.
Processed : Metro Manila NCR Marikina City Amang Rodriguez Memorial Medical Center
Processed : Metro Manila NCR Marikina City Amang Rodriguez Memorial Medical Center
Processed : Metro Manila NCR Quezon City Hero Laboratories
Processed : Metro Manila NCR Taguig City Lifecore Bio Integrative Inc.
Processed : Metro Manila NCR Muntinlupa City Ospital ng Muntinlupa
Processed : Metro Manila NCR City of Manila Supercare Medical Services, Inc.
Processed : Metro Manila NCR Pasay City JT Cenica Medical Health System
Processed : Metro Manila NCR Quezon City New World Diagnostics Inc. Premium Medical Branch
Processed : Metro Manila NCR Las Piñas City A Star Laboratorie

Processed : Pampanga Region 3 Mabalacat City The Medical City Clark Inc.
Processed : Bataan Region 3 Municipality of Mariveles Mariveles District Hospital
Processed : Bulacan Region 3 Municipality of Sta. Maria Multi-Lifecare Foundation Molecular Laboratory
Processed : Bataan Region 3 Municipality of Mariveles Mariveles District Hospital
Processed : Nueva Ecija Region 3 Cabanatuan City Dr. Paulino J. Garcia Memorial Research and Medical Center
Processed : Bulacan Region 3 San Jose Del Monte City Skyline Hospital and Medical Center
Processed : Zambales Region 3 Olongapo City Olongapo City Molecular Diagnostic Laboratory
Processed : Pampanga Region 3 Municipality of Guagua Pampanga Medical Specialist Hospital
Processed : Pampanga Region 3 San Fernando City Mother Teresa of Calcutta Medical Center
Processed : Pampanga Region 3 San Fernando City Mother Teresa of Calcutta Medical Center
Processed : Pampanga Region 3 Municipality of Guagua Pampanga Molecular Laboratory
Processed : Pampanga R

Processed : Cebu Region 7 Cebu City Chong Hua Hospital
Processed : Bohol Region 7 Tagbilaran City Bohol Containerized PCR Laboratory
Processed : Cebu Region 7 Cebu City Cebu TB Reference Laboratory
Processed : Cebu Region 7 Cebu City Cebu TB Reference Laboratory
Processed : Negros Oriental Region 7 Dumaguete City Negros Oriental Provincial Hospital
Processed : Cebu Region 7 Cebu City Perpetual Succour Hospital of Cebu, Inc.
Processed : Cebu Region 7 Cebu City BioPath Clinical Diagnostic, Inc. - AFRIMS
Processed : Cebu Region 7 Cebu City BioPath Clinical Diagnostic, Inc. - AFRIMS PCR laboratory
Processed : Cebu Region 7 Cebu City RHS 7 Molecular Laboratory (PNP)
Processed : Cebu Region 7 Cebu City Perpetual Succour Hospital of Cebu, Inc.
Processed : Cebu Region 7 Cebu City Chong Hua Hospital
Processed : Cebu Region 7 Cebu City Cebu Doctors University Hospital, Inc.
Processed : Cebu Region 7 Dumaguete City Silliman University Medical Center Foundation, Inc.
Processed : Leyte Region 8 Tac

In [20]:
df.head()

,region,province,city,name_of_facility,ownership,type_of_testing,license_validity,index,current_distance,gmaps_url,mapping_address,lat,lng
0,NCR,Metro Manila,Muntilupa City,Asian Hospital and Medical Center,PRIVATE,rRT PCR,01/01/22 - 12/31/22,0,Click the compass to find out!,https://www.google.com/maps/dir/?api=1,Metro Manila NCR Muntilupa City Asian Hospital...,14.413465,121.043533
1,NCR,Metro Manila,City of Manila,Chinese General Hospital,PRIVATE,rRT PCR,01/08/22 - 12/31/22,1,Click the compass to find out!,https://www.google.com/maps/dir/?api=1,Metro Manila NCR City of Manila Chinese Genera...,14.625585,120.988647
2,NCR,Metro Manila,Mandaluyong City,Detoxicare Molecular Diagnostics Laboratory,PRIVATE,rRT PCR,01/01/22 - 12/31/22,2,Click the compass to find out!,https://www.google.com/maps/dir/?api=1,Metro Manila NCR Mandaluyong City Detoxicare M...,14.591336,121.032869
3,NCR,Metro Manila,Caloocan City,Dr. Jose N. Rodriguez Memorial Hospital and Sa...,PUBLIC,Cartridge - Based PCR,01/01/22 - 12/31/22,3,Click the compass to find out!,https://www.google.com/maps/dir/?api=1,Metro Manila NCR Caloocan City Dr. Jose N. Rod...,14.766724,121.064842
4,NCR,Metro Manila,Caloocan City,Dr. Jose N. Rodriguez Memorial Hospital and Sa...,PUBLIC,rRT PCR,01/01/22 - 12/31/22,4,Click the compass to find out!,https://www.google.com/maps/dir/?api=1,Metro Manila NCR Caloocan City Dr. Jose N. Rod...,14.766724,121.064842


In [21]:
df.shape

(340, 13)

In [22]:
#remove duplicated rows
df = df.drop_duplicates(keep='first')

In [23]:
df.shape

(340, 13)

In [24]:
no_coord_df = df[(df['lat'] == "NaN") | (df['lng'] == "NaN")]
no_coord_df.shape

(0, 13)

In [25]:
coord_df = df[(df['lat'] != "NaN") & (df['lng'] != "NaN")]
coord_df.shape

(340, 13)

In [26]:
coord_df.to_csv ("mapping_data_covidlab.csv", index = False)

In [27]:
no_coord_df.to_csv ("no_mapping_data_covidlab.csv", index = False)